# Using the $k$-Nearest Neighbor Algorithm

In this notebook, we'll look at how we can use existing Python libraries to implement the $k$-nearest neighbor algorithm for the iris dataset. 

## Setting Things Up

First, we need to import the libraries that we'll need.

Pandas is a library for high-performance and easy to use data structures. Using this library makes machine learning both simpler and faster. Pandas give you many built-in tools to work with datasets and it handles dates and times very well!

Numpy is a common and fundamental package for scientific computing. Many other libraries make use of numpy. The name comes from the words "numerical" and "Python" mashed together.

Scikit-learn is a library for machine learning in Python. It does the work of implementing machine learning algorithms for us, and also contains commonly used datasets. Although we'll be using other things from Scikit-learn later, to start we only need to load the iris dataset.

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets

Next, we'll load the data from the iris data set. In order to more easily work with this data, we'll store it as a pandas dataframe. The second line gathers the data from the iris dataset, and converts it into a pandas dataframe. This dataframe, iris_df, is what we'll be working with from now on. You can find more information about the iris dataset [here](https://en.wikipedia.org/wiki/Iris_flower_data_set).

In [ ]:
iris = datasets.load_iris()
iris_df = pd.DataFrame(data = np.c_[iris['data'], iris['target']], columns = iris['feature_names'] + ['Species'])

Now that we've got out data set up, we're ready to start experimenting with it!

## Initial Investigation

Before we apply the $k$-nearest neighbor algorithm to our dataset, let's do some preliminary investigation. This will serve to verify that our data seems to be set up correctly, and also give us a rough idea of how the data is distributed.

First, let's check out how much data we have.

In [ ]:
iris_df.shape

In [ ]:
iris_df.columns.values

We have 150 data points, and each point has five coordinates. Four of these are the features of the flower: sepal length, sepal width, petal length, and petal width. In this example, the columns of the data come labelled with these names, but in data you find elsewhere that may not be automatically true. 

The last coordinate is the species of the flower. Let's see what values the species of the flower can take.

In [ ]:
print(iris_df['Species'].values)

This should seem a little strange - these should be species, not numbers!

This is actually okay. Here, instead of storing a string with the name of each species, we store an integer corresponding to the species. The integer 0 corresponds to Iris-setosa, 1 corresponds to Iris-versicolor, and 2 corresponds to Iris-virginica.

Next, let's look at some summary statistics for the dataset.

In [ ]:
iris_df.describe()

Looking at these statistics for the features is interesting and useful information. However, averaging the species number for the all of the data points doesn't really make sense, so we can just ignore that column.

Next, let's do some plotting so we can see what our data looks like. In order to do this, we need to import a couple more libraries.

Matplotlib is a plotting library that aims to make plotting in Python easy. Seaborn is a library for statistical data visualization, which is based on matplotlib and makes much prettier plots.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

The pairplot function will take two variables at a time, and plot the values of the datapoints for those to variables.

The first parameter that we give this function is our dataframe, iris_df.

The second parameter tells it which variables we want to plot. By entering iris_df.columns[:-1], we take all of the variables of iris_df except for the last one, which is the species. (The negative one (-1) tells the columns function to drop the last column from the end.) This way, we plot all of the features against each other, but not against the species. 

The last parameter tells the function to color the data points according to the their "Species" values. This way, we can easily see which species each data point is across all of the plots, and we can see how they are clustered.

In [ ]:
plt.figure()
sns.pairplot(iris_df, vars=iris_df.columns[:-1], hue = "Species")
plt.show()

From our plots, we can see that the species do seem to be grouped according to these measurements. It seems like using the $k$-nearest neighbor algorithm will work well for this dataset!

# $k$-Nearest Neighbor

Using our preliminary data analysis, we expect that we'll be able to use the $k$-nearest neighbor algorithm effectively for this dataset, so that's what we'll do next.

Since we want to see how the features can be used to predict the species of each iris, we'll separate our data into the features and the species.

In [ ]:
X = iris_df[iris.feature_names]
y = iris_df["Species"]

Next, we'll split our data into a training set and a testing set. Fortunately, we don't have to do this by hand; there's a scikit-learn function that can do it for us!

For the first parameter of train_test_split, we use the features. For the second parameter, we use the species, which is what we're trying to predict.

The third parameter tells the function how large to make the test set. By stating test_size=0.2, we're deciding that 20% of the data will be used for testing.

The last parameter determines how the data set is split. Here, we specify a random_state, to make sure that we'll get the same split each time we run this code. If we change 0 to another number, we'd get a different split. If we left off this parameter, the dataset would have a different random split each time we ran the code! This is often desirable, but for now, when we're just getting started, we'd like more predictable results.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Next, we'll import some more functions from scikit-learn. We now import the $k$-nearest neighbor classifier, along with some additional functions which will help us evaluate how well our classifier is performing.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

Next we build our classifier. To start out, we'll use the $3$-nearest neighbors. Later, we'll experiment with different values for $k$. After we've chosen our classifier, we train the classifier using our training set. These two short lines of code do all of the work of the $k$-nearest neighbor algorithm, which is pretty incredible!

The first line tells the computer, "Build a classifier model using KNeighbors", and the second line says, "Fit the model on my training data with X_train as the input and y_train as the classification." The computer will then build the k-nearest neighbors classifier using all of this information.

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train, y_train)

Now that we've built and trained our classifier, we use it to predict the species for our testing sets. This line of code says, "Use the classifier we built on our test data, X_test, and make a prediction y_pred for the species."

In [ ]:
y_pred = classifier.predict(X_test)

We've used our classifier to predict the species values for our testing set, and we'd like to compare these to the known species values for the testing set. To easily see how many data points were classified correctly, we use a function a find the confusion matrix. The rows are the actual species and the columns are the predicted species: we want all the non-zero entries to be on the diagonal of this matrix.

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

We see that only one data point from the test set was classified incorrectly - that seems pretty good! One of the flowers that was actually in species 2 was predicted to be in species 3 (there is a one in the second row, third column). 

We can also compute the accuracy score, which tells us what proportion of data points were predicted correctly.

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

An accuracy score of 1 is perfect, so .966... is good! This means that 96.66...% of the data points from the testing set were classified correctly.

# Finding the Optimal Value for $k$

It seems like the $3$-nearest neighbor algorithm performs well for classifying irises, but is there a value of $k$ that would work even better? We'll now experiment with different values of $k$, to try to figure out the best possible value.

We'll test values $k=1,3,5,7,...,49$, so we take the range from $1$ to $50$, going up by $2$ each time. For each value of $k$, we'll compute a cross-validation score, which will tell us the proportion of data points which are classified correctly.

In [ ]:
k_list = list(range(1,50,2))
cv_scores = []

Now for the actual testing! For each value of $k$, we construct a $k$-nearest neighbor classifier. Then, we train our classifier. There's some potential for problems here - what if we got really lucky or unlucky with our split between training and testing set, and wind up with results that don't accurately reflect the accuracy of our algorithm?

Using cross-validation fixes this problem. Instead of training and testing once on the given split, the cross_val_score function creates 10 variations on the training and testing sets, and evaluates the accuracy of the classifier for each of these 10 splits. That way, we're automatically evaluating the $k$-nearest neighbor algorithm on 10 different training and testing splits, without having to perform these splits ourselves!

Finally, we compute the average of the accuracies for all 10 trials, and record this as the overall score for the current value of $k$.

At the end of this process, we have an array with an accuracy score for each value of $k$.



In [ ]:
for k in k_list:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv = 10, scoring = 'accuracy')
    cv_scores.append(scores.mean())

We'll plot the scores that we found against the $k$ values, so we can see which value of $k$ gives the most accurate results.

In [ ]:
plt.figure()
plt.title('Performance of K Nearest Neighbors Algorithm')
plt.xlabel('Number of Neighbors K')
plt.ylabel('Accuracy Score')
plt.plot(k_list, cv_scores)

plt.show()

From our graph, it looks like the $k$-nearest neighbor algorithm performs best for $k=9$ or $k=11$, for our dataset. We can verify this, by having Python check which value of $k$ gives the highest accuracy score.

In [ ]:
best_k = k_list[cv_scores.index(max(cv_scores))]
print(best_k)

We see that $k=9$ maximizes the accuracy score for the $k$-nearest neighbor algorithm for the iris dataset. Yay!